In [1]:
from itertools import product

import numpy as np

from IPython.display import display

from sympy import symbols, Eq
from sympy import zeros, diag, Matrix

import qlp.eqn_converter as qlp

In [2]:
a1, a2, b1, b2, x1, x2, s1, s2, p, c1, c2 = symbols(
    "a1, a2, b1, b2, x1, x2, s1, s2, p, c1, c2"
)
subs = {
    "a1": "0.456",
    "a2": "0.1",
    "b1": "10",
    "b2": "-2.1",
    "c1": 2,
    "c2": 0.5,
    "p": 100,
}

eqns = [
    a1 * x1 <= b1,
    a2 * x1 + a1 * x2 <= b2,
]

x = Matrix([x1, x2])
s = Matrix([s1, s2])
c = Matrix([c1, c2])

In [3]:
a, b, d = qlp.eqns_to_matrix(eqns, x)

mm = a @ x + b + d @ s
print("A x - b + D s")
display(mm)

print("Argument of (3)")
c.T @ x - mm.T @ mm

A x - b + D s


Matrix([
[        a1*x1 - b1 + s1],
[a1*x2 + a2*x1 - b2 + s2]])

Argument of (3)


Matrix([[c1*x1 + c2*x2 - (a1*x1 - b1 + s1)**2 - (a1*x2 + a2*x1 - b2 + s2)**2]])

In [4]:
print(subs)
print("A x - b + D s after rescaling")
qlp.rescale_expressions(mm, subs)

{'a1': '0.456', 'a2': '0.1', 'b1': '10', 'b2': '-2.1', 'c1': 2, 'c2': 0.5, 'p': 100}
A x - b + D s after rescaling
Multipying by 1000


Matrix([
[        s1 + 456*x1 - 10000],
[s2 + 100*x1 + 456*x2 + 2100]])

In [5]:
print("eq (11) with Nb = 2")
q = qlp.get_bit_map(len(x) + len(s), 2)
q

eq (11) with Nb = 2


array([[1, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 2]])

In [6]:
alpha = zeros(rows=a.rows, cols=a.cols + d.cols)

for ncol, nrow in product(range(a.rows), range(a.cols)):
    alpha[ncol, nrow] = a[ncol, nrow]

for ncol, nrow in product(range(a.rows), range(d.cols)):
    alpha[ncol, a.cols + nrow] = d[ncol, nrow]

alpha_q = alpha @ q

print("alpha: eq (14)")
display(alpha)
print("alpha.q:")
display(alpha_q)

alpha: eq (14)


Matrix([
[a1,  0, 1, 0],
[a2, a1, 0, 1]])

alpha.q:


Matrix([
[a1, 2*a1,  0,    0, 1, 2, 0, 0],
[a2, 2*a2, a1, 2*a1, 0, 0, 1, 2]])

In [7]:
gamma = zeros(rows=c.rows + b.rows, cols=1)

for nrow in range(c.rows):
    gamma[nrow] = c[nrow]

gamma_q = gamma.T @ q

print("gamma: eq (12)")
display(gamma)

print("gamma.q")
display(gamma_q)

gamma: eq (12)


Matrix([
[c1],
[c2],
[ 0],
[ 0]])

gamma.q


Matrix([[c1, 2*c1, c2, 2*c2, 0, 0, 0, 0]])

In [8]:
print("b.alpha.q: eq (15)")
display(b.T @ alpha_q)

print("omega: eq (17)")
omega = diag(*gamma_q) + alpha_q.T @ alpha_q - 2 * p * diag(*b.T @ alpha_q)
omega

b.alpha.q: eq (15)


Matrix([[-a1*b1 - a2*b2, -2*a1*b1 - 2*a2*b2, -a1*b2, -2*a1*b2, -b1, -2*b1, -b2, -2*b2]])

omega: eq (17)


Matrix([
[a1**2 + a2**2 + c1 - 2*p*(-a1*b1 - a2*b2),                                   2*a1**2 + 2*a2**2,                  a1*a2,                    2*a1*a2,         a1,       2*a1,         a2,       2*a2],
[                        2*a1**2 + 2*a2**2, 4*a1**2 + 4*a2**2 + 2*c1 - 2*p*(-2*a1*b1 - 2*a2*b2),                2*a1*a2,                    4*a1*a2,       2*a1,       4*a1,       2*a2,       4*a2],
[                                    a1*a2,                                             2*a1*a2, a1**2 + 2*a1*b2*p + c2,                    2*a1**2,          0,          0,         a1,       2*a1],
[                                  2*a1*a2,                                             4*a1*a2,                2*a1**2, 4*a1**2 + 4*a1*b2*p + 2*c2,          0,          0,       2*a1,       4*a1],
[                                       a1,                                                2*a1,                      0,                          0, 2*b1*p + 1,          2,          0,          0],
[

In [9]:
print("omega after substituion")
omega_subs = np.array(qlp.rescale_expressions(omega, subs))
omega_subs

omega after substituion
Multipying by 1000


array([[870000219936, 435872, 45600, 91200, 456, 912, 100, 200],
       [435872, 1740000875744, 91200, 182400, 912, 1824, 200, 400],
       [45600, 91200, -191519791564, 415872, 0, 0, 456, 912],
       [91200, 182400, 415872, -383039167256, 0, 0, 912, 1824],
       [456, 912, 0, 0, 2000000001, 2, 0, 0],
       [912, 1824, 0, 0, 2, 4000000004, 0, 0],
       [100, 200, 456, 912, 0, 0, -419999999, 2],
       [200, 400, 912, 1824, 0, 0, 2, -839999996]], dtype=object)